# Data Preprocessing
We use Kaggle's <a href='https://www.kaggle.com/datasets/gauravtopre/bank-customer-churn-dataset'>Bank Customer Churn</a> dataset to build a **semi-supervised model for label prediction**.

We will train various models using Germany data, and predict on Spain customers to see how well the models generalize.

This script processes the data for downstream modelling.

In [19]:
import random
import datetime

import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport

from scipy import stats

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split


SEED = 2023
np.random.seed(SEED)
random.seed(SEED)

## 1 Load data

In [20]:
DATA_PATH = '../../data'

In [21]:
df = pd.read_csv(f'{DATA_PATH}/raw/Bank Customer Churn Prediction.csv')

# Generate profile report
profile = ProfileReport(df, title="Profiling Report")
# profile

In [22]:
df.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## 2 Data Preperation

In [23]:
# Split the df by country
df_germany = df[df['country']=='Germany']
df_spain = df[df['country']=='Spain']
df_france = df[df['country']=='France']

datasets = {'germany':df_germany, 'spain':df_spain, 'france':df_france}

In [24]:
# Clean up the binary features into boolean first
binary_values = ['Y','y',1,'Yes']
binary_cols = ['active_member','credit_card']

for dataset in datasets.values():
    for col in binary_cols:
        dataset[col] = dataset[col].apply(lambda x: True if x in binary_values else False)

datasets['spain'].head()

/var/folders/0g/_vw0ydz94_x5cpswktv47t9h0000gn/T/ipykernel_37368/2212832797.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[col] = dataset[col].apply(lambda x: True if x in binary_values else False)


,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
1,15647311,608,Spain,Female,41,1,83807.86,1,False,True,112542.58,0
4,15737888,850,Spain,Female,43,2,125510.82,1,True,True,79084.10,0
5,15574012,645,Spain,Male,44,8,113755.78,2,True,False,149756.71,1
11,15737173,497,Spain,Male,24,3,0.00,2,True,False,76390.01,0
14,15600882,635,Spain,Female,35,7,0.00,2,True,True,65951.65,0


In [12]:
# Mean and zero imputation

numeric_features_impute_mean_rescale = ['credit_score','age','tenure','balance']
numeric_features_impute_zero_rescale = ['products_number','estimated_salary']
categorical_features_encode = ['gender','credit_card','active_member']
other_features = ['customer_id','country','churn']
all_cols = numeric_features_impute_mean_rescale + numeric_features_impute_zero_rescale + categorical_features_encode+ other_features


# Specify the transformations

trf_mean_rescale = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),
                                #    ('scaler', MinMaxScaler()) # Not necessary at this stage, since all units incl currency are the same for this dataset.
                                   ])

trf_zero_rescale = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value=0)),
                                #    ('scaler', MinMaxScaler()) # Not necessary at this stage, since all units incl currency are the same for this dataset.
                                   ])

trf_cat_encoder = Pipeline(steps=[
    ('encoder', OneHotEncoder(drop='if_binary')),
])


# Combine transformers into a ColumnTransformer
country_preprocessor = ColumnTransformer(
    transformers=[
        ('num_mean_rescale', trf_mean_rescale, numeric_features_impute_mean_rescale),
        ('num_zero_resacle', trf_zero_rescale, numeric_features_impute_zero_rescale),
        ('cat_encoder', trf_cat_encoder, categorical_features_encode),
        ('others', 'passthrough', other_features)
    ],
    remainder='drop'
    )


# Fit and transform the data seperately by country
df_germany_transformed = country_preprocessor.fit_transform(df_germany)
df_germany_transformed = pd.DataFrame(df_germany_transformed, columns=all_cols)

df_spain_transformed = country_preprocessor.fit_transform(df_spain)
df_spain_transformed = pd.DataFrame(df_spain_transformed, columns=all_cols)


## 3 Exploratory Data Analysis
We compare Germany and Spain datasets, to see if there is any statisically significant differences in both countries.

In [14]:
# Compare Germany and Spain
df1, df2 = df_germany_transformed, df_spain_transformed

### 3.1 t-test on numeric columns
We perform independent samples t-test to see if we can combine the the datasets from Germany and Spain.

If $\text{p-value} < \alpha = 0.05$ for $\text{feature}_i$, both countries are NOT significant different on $\text{feature}_i$ and therefore can be combined.

In [15]:
# Set alpha
alpha = 0.05

t_test_cols = ['credit_score','age','tenure','balance','products_number','estimated_salary']

# Check if the cols in the different datasets are significantly different
sig_diff_features, not_sig_diff_features = [], []

for col in t_test_cols:
    result = stats.ttest_ind(df1[col].to_list(), df2[col].to_list(),equal_var=False, random_state=SEED)
    p_val = result.pvalue
    if p_val >= alpha:
        not_sig_diff_features.append(col)
    else:
        sig_diff_features.append(col)

print(f"{len(not_sig_diff_features)} out of {len(t_test_cols)} features are NOT significantly different.")
print(f"NOT significanly different features: {not_sig_diff_features}")
print(f"Significanly different features: {sig_diff_features}")

4 out of 6 features are NOT significantly different.
NOT significanly different features: ['credit_score', 'tenure', 'products_number', 'estimated_salary']
Significanly different features: ['age', 'balance']


### 3.2 Chi-square test for independence on categorical columms

In [16]:
chi_sq_cols = ['gender','credit_card','active_member','churn']

# Combine 2 countries' data
combined_df = df1._append(df2)

# Check if the cols in the different datasets are significantly different
sig_diff_features, not_sig_diff_features = [], []

for col in chi_sq_cols:
    # Construct the contingency table
    pt = pd.pivot_table(combined_df, values='customer_id', index='country', columns=[col], aggfunc='count')
    # Chi-square test
    chi2, p_val, dof, expected = stats.chi2_contingency(pt)
    if p_val >= alpha:
        not_sig_diff_features.append(col)
    else:
        sig_diff_features.append(col)

print(f"{len(not_sig_diff_features)} out of {len(t_test_cols)} features are NOT significantly different.")
print(f"NOT significanly different features: {not_sig_diff_features}")
print(f"Significanly different features: {sig_diff_features}")

1 out of 6 features are NOT significantly different.
NOT significanly different features: ['credit_card']
Significanly different features: ['gender', 'active_member', 'churn']


### 4 Prepare dataset for training the model
1. **Keep all features.** Ideally, we should only combine the datasets on columns that are not significantly different. However, since there only a few columns, we choose to keep all of them to preseve as much information as possible
2. **Train-test split.**</br>
   a. Train set: 80% of Germany's data using stratified split with random shuffling</br>
   b. Validation set: 20% of Germany's data using stratified split with random shuffling</br>
   c. Test set: 100% of Spain's data

In [17]:
target = 'churn'

# Split Germany's data into training and validation sets
X = df_germany_transformed.loc[:, ~df_germany_transformed.columns.isin([target])]
y = df_germany_transformed.loc[:, target].astype(int)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=SEED)
X_test = df_spain_transformed.loc[:, ~df_spain_transformed.columns.isin([target])]
y_test = df_spain_transformed.loc[:, target]

In [18]:
# Output pickle files
X_train.to_pickle(f"{DATA_PATH}/processed/X_train.pkl")
X_val.to_pickle(f"{DATA_PATH}/processed/X_val.pkl")
X_test.to_pickle(f"{DATA_PATH}/processed/X_test.pkl")

y_train.to_pickle(f"{DATA_PATH}/processed/y_train.pkl")
y_val.to_pickle(f"{DATA_PATH}/processed/y_val.pkl")
y_test.to_pickle(f"{DATA_PATH}/processed/y_test.pkl")